In [9]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
# import keras_metrics as km
# from keras import regularizers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
import math
#import vidaug.augmentors as va
from sklearn.utils import shuffle


#모델
new_model=tf.keras.models.load_model('./4aug_epoch25.h5')
#new_model.summary()
#카메라 키기 
def camera():
        import cv2
        import numpy as np
        import time
        from time import sleep

        img_counter = 0
        frame_set = []
        start_time = time.time()
        pre_test = "hi"

        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)


        while True:
            ret, image = cap.read()
            frame = cv2.flip(image, 1)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('image', frame)
           
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if time.time() - start_time >= 0.2:  # <---- Check if 0.2 sec passed
                image = cv2.resize(frame, (64, 64))
                frame_set.append(image)
                print("{} written!".format(img_counter))
                start_time = time.time()
            img_counter += 1
            if len(frame_set) >= 17:
                del frame_set[0]
            if len(frame_set)==16:
                predict = new_model.predict(np.reshape(np.array(frame_set)/255,(1,16,64,64,3)))
                predict = np.argmax(predict, axis = 1)[0]
                print(predict)
                
                
                if predict==1:
                    lab6.config(text="앗! 아이가 위험 물질을 잡았어요! ")
                    lab6.configure(bg='#FFB399')
                    print("앗! 아이가 위험 물질을 잡았어요! ") 
                    if(pre_test!="play"):
                        play()
                        pre_test="play"
                        print(pre_test)
                elif predict ==0 or predict==2 or predict ==5:
                    if predict==0: 
                        lab6.config(text="넘어졌어요! 위험해요!!!")
                        lab6.configure(bg='#FFB399')
                        print("앗! 넘어졌어요! 위험해요")
                    elif predict==2 :
                        lab6.config(text="무언가를 삼켰어요! 위험해요!!!")
                        lab6.configure(bg='#FFB399')
                        print("앗! 무언가를 삼켰어요! 위험해요")
                    else:
                        lab6.config(text="아이가 어디를 기어올라가요! 위험해요!!! ")
                        lab6.configure(bg='#FFB399')
                        print("앗! 아이가 어디를 기어올라가요! 위험해요! ")       
                    if(pre_test!="callsms"):
                        #callsms()
                        pre_test="callsms"
                        print(pre_test)
                else:
                    lab6.config(text="아이가 안전해요!")
                    lab6.configure(bg='#FFB399')
                    print("아이가 안전해요!")
                    if(pre_test!="safe"):
                        pre_test="safe"
                        print(pre_test)     


            

                        
#오디오 기록
def audio():
    
    import pyaudio
    import wave
    po = pyaudio.PyAudio()
    

    for index in range(po.get_device_count()):
        desc = po.get_device_info_by_index(index)

        # if desc["name"] == "record":

        print("DEVICE: %s  INDEX:  %s  RATE:  %s " % (desc["name"], index, int(desc["defaultSampleRate"])))
        
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    CHUNK = 1024
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "file.wav"
    
    audio = pyaudio.PyAudio()
    # start Recording
    stream = audio.open(format=pyaudio.paInt16,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        input_device_index=1,
                        frames_per_buffer=CHUNK)

    print("recording...")

    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)

        frames.append(data)

    print("finished recording")
    lab8.config(text="녹음되었습니다!")
    lab8.configure(bg='#FFCCCC')

    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()


#문자 전송( 버튼 클릭 시 )
def callsms1(num):
    import sys
    from sdk.api.message import Message
    from sdk.exceptions import CoolsmsException
    api_key = "NCSIAKAHOQDSODRN"
    api_secret = "Y42NPXZBUAVLJPO7PIKMWKWNNTHGWQZS"
    ## 4 params(to, from, type, text) are mandatory. must be filled
    params = dict()
    params['type'] = 'sms' # Message type ( sms, lms, mms, ata )
    params['to'] = ent.get()# Recipients Number '01000000000,01000000001'
    params['from'] = '01072108042' # Sender number
    params['text'] = '아이가 위험합니다!!!' # Message
    cool = Message(api_key, api_secret)
    try:
        response = cool.send(params)
        print("Success Count : %s" % response['success_count'])
        print("Error Count : %s" % response['error_count'])
        print("Group ID : %s" % response['group_id'])
    except CoolsmsException as e:
        print("Error Code : %s" % e.code)
        print("Error Message : %s" % e.msg)
#    sys.exit()
    
#위험 행동 시 
def callsms():
    import sys
    from sdk.api.message import Message
    from sdk.exceptions import CoolsmsException
    api_key = "NCSIAKAHOQDSODRN"
    api_secret = "Y42NPXZBUAVLJPO7PIKMWKWNNTHGWQZS"
    ## 4 params(to, from, type, text) are mandatory. must be filled
    params = dict()
    params['type'] = 'sms' # Message type ( sms, lms, mms, ata )
    params['to'] = ent.get()# Recipients Number '01000000000,01000000001'
    params['from'] = '01072108042' # Sender number
    params['text'] = '아이가 위험합니다!' # Message
    cool = Message(api_key, api_secret)
    try:
        response = cool.send(params)
        print("Success Count : %s" % response['success_count'])
        print("Error Count : %s" % response['error_count'])
        print("Group ID : %s" % response['group_id'])
    except CoolsmsException as e:
        print("Error Code : %s" % e.code)
        print("Error Message : %s" % e.msg)
    sys.exit()

                
                        
#오디오 재생코드                
def play():
  
    import pyaudio
    import wave
    CHUNK = 1024

    path = './file.wav'

    with wave.open(path, 'rb') as f:
        p = pyaudio.PyAudio()
        stream = p.open(format=p.get_format_from_width(f.getsampwidth()),
                        channels=f.getnchannels(),
                        rate=f.getframerate(),
                        output=True)

        data = f.readframes(CHUNK)
        while data:
            stream.write(data)
            data = f.readframes(CHUNK)

        stream.stop_stream()
        stream.close()

        p.terminate()
        
#GUI구현
from tkinter import *
import tkinter.messagebox

# 창 생성
win = Tk() 
win.geometry("1350x750")
win.title("AI-KU System")
win.option_add("*Font", "맑은고딕 15")
win.configure(bg='#FFEAD0')

lab_logo = Label(win)
lab_logo.place(x=50, y=50)
img_logo = PhotoImage(file='./iKu.png', master=win)
lab_logo.config(image=img_logo)
lab_logo.config(bg='#FFEAD0')


# 카메라 라벨
lab0 = Label(win)
lab0.config(text="<전화번호와 오디오를 저장한 후 카메라를 실행하세요>")
lab0.configure(bg='#FFB399')
lab0.place(x=800, y=100)

# 버튼
btn0 = Button(win, text="Camera")
btn0.config(width=40, height=40)
btn0.place(x=800, y=140)
img0 = PhotoImage(file='./camera.png', master=win)
img0 = img0.subsample(25)
btn0.config(image=img0)
btn0.config(command=camera)

# 라벨
lab1 = Label(win)
lab1.config(text="번호를 입력하세요: ")
lab1.configure(bg='#FFB399')
lab1.place(x=800, y=250)

# 전화 번호 입력창
ent = Entry(win)
ent.insert(0, "-없이 숫자만 가능")
def clear(event):
    if ent.get() == "-없이 숫자만 가능":
        ent.delete(0, len(ent.get()))
ent.bind("<Button-1>", clear)
ent.place(x=800, y=280)
ent.configure(bg='#FFDAAB')



def Msgbox():
    tkinter.messagebox.showerror("메시지", "숫자만 가능합니다.")


def alert():
    regex = re.compile("^[0-9]{11}$")
    if regex.findall(ent.get()):
        lab2.config(text="저장되었습니다!")
        lab2.configure(bg='#FFCCCC')
    else:
        Msgbox()


def alert1():
    regex = re.compile("^[0-9]{11}$")
    if regex.findall(ent.get()):
        lab2.config(text="수정되었습니다!")
        lab2.configure(bg='#FFCCCC')
    else:
        Msgbox()
    
# 저장 버튼
btn = Button(win, text="Save")
btn.config(command=alert)
btn.config(width=40, height=40)
btn.place(x=800, y=320)
img = PhotoImage(file='./check.png', master=win)
img = img.subsample(25)
btn.config(image=img)

# 수정 버튼
btn_ = Button(win, text="Change")
btn_.config(command=alert1)
btn_.config(width=40, height=40)
btn_.place(x=880, y=320)
img_ = PhotoImage(file='./exchange.png', master=win)
img_ = img_.subsample(25)
btn_.config(image=img_)

# 메시지 라벨
lab2 = Label(win)
lab2.place(x=800, y=370)
lab2.configure(bg='#FFEAD0')

def button_click(): # This button_click function is used for send button in Tkinter GUI window.
    sender = callsms1(ent.get())

#q전송 버튼
sendBtn = Button(win, text="SEND", command=button_click,fg="white",cursor='hand2')
sendBtn.place(x=960,y=320)
img_send = PhotoImage(file='./send.png', master=win)
img_send = img_send.subsample(25)
sendBtn.config(image=img_send)
sendBtn.config(width=40, height=40)

# 라벨
lab3 = Label(win)
lab3.config(text="녹음을 시작하세요: (5초 동안)")
lab3.configure(bg='#FFB399')
lab3.place(x=800, y=450)
    
# 녹음 시작 버튼
btn1 = Button(win, text="Start")
btn1.config(width=40, height=40)
btn1.place(x=800, y=490)
img1 = PhotoImage(file='./microphone.png', master=win)
img1 = img1.subsample(25)
btn1.config(image=img1)
btn1.config(command=audio)

lab8 = Label(win)
lab8.place(x=800, y=540)
lab8.configure(bg='#FFEAD0')

lab4 = Label(win)
lab4.config(text="화면 끄기")
lab4.configure(bg='#FFB399')
lab4.place(x=800, y=620)

#lab6 = Label(win)
#lab6.place(x=300, y=600)
#lab6.config(text="안전 행동!")
#lab6.configure(bg='#FFB399')
# exit
def close_window():
    win.destroy()

btn2 = Button(win, text="Exit")
btn2.config(command=close_window)
btn2.place(x=800, y=660)
img2 = PhotoImage(file='./exit.png', master=win)
img2 = img2.subsample(25)
btn2.config(width=40, height=40)
btn2.config(image=img2)

lab6 = Label(win)
lab6.place(x=270, y=600)
lab6.configure(bg='#FFEAD0')
#lab6.configure(bg='#FFB399')
#lab6.configure(text='앗! 아이가 위험 물질을 잡았어요!')

win.mainloop()  # 창 실행